In [1]:
from MyModule import *

In [2]:
data = pd.read_excel(r'C:\Users\gbouvier\Documents\census.xlsx', sheet_name='Study')

In [3]:
%matplotlib qt
warnings.filterwarnings("ignore", category=UserWarning)
%run multi_animal_function.ipynb

# Spike number

In [4]:
animal = 'animal21_a17d1s1'

for neuron in range(rotation_data[animal]['Nclust']):
    neuron += 1
    print(f"Neuron {neuron} : {'{:,.0f}'.format(len(rotation_data[animal]['c_class'][rotation_data[animal]['c_class'][:,0]==neuron,1])).replace(',', ' ')}") 

NameError: name 'rotation_data' is not defined

# Autocorr figure

In [ ]:
%matplotlib qt

max_lag = 1000

bin_size = 0.5

animal = 'animal21_a17d1s1'

neuron = 20

autocorr = []

c_class = rotation_data[animal]['c_class']

for spike in tqdm(c_class[c_class[:,0]==neuron,1]):
    for spike2 in c_class[c_class[:,0]==neuron,1]:
        if spike != spike2 and abs(spike - spike2) <= max_lag:
            autocorr.append(spike - spike2)


plt.figure()
plt.hist(autocorr, bins=np.arange(-max_lag, max_lag, bin_size))
plt.margins(x=0)
plt.xlabel('Lag (ms)')
plt.ylabel('Number of spikes')
plt.title(f"Autocorrelogram of neuron {neuron}, {animal}")
plt.show()

# ISI figure

In [ ]:
isi = np.diff(rotation_data[animal]['c_class'][rotation_data[animal]['c_class'][:,0]==neuron,1])


plt.figure()
plt.hist(isi, bins=8000)
plt.margins(x=0)
plt.ylabel('Counts')
plt.xlabel('ISI (ms)')
plt.title(f"ISI of neuron {neuron}, {rotation_data[animal]['exp_id']}")
# plt.xlim(0,200)
plt.show()

# Spike Waveforms

In [ ]:
# df = pd.read_excel(r'c:\Users\gbouvier\Documents\census.xlsx')
# raw_data_path = df['raw_data_path'][df['exp_id']==animal].iloc[0]

# times_polytrodeAll = scipy.io.loadmat(os.path.join(raw_data_path, 'times_polytrodeAll.mat'))
# # suaWf = times_polytrodeAll['clusWFmua']
# suaWf = times_polytrodeAll['clusWFs']


# for unit in range(len(suaWf)):
#     max_amplitude = max(np.concatenate([abs(suaWf[unit][:,trace]) for trace in range(suaWf[unit].shape[1])]))
#     iteration = 0
#     plt.figure(figsize=(4,8))

#     for trace in range(suaWf[unit].shape[1]):
#         if np.std(suaWf[unit][:,trace])>np.std(np.concatenate([abs(suaWf[unit][:,trace]) for trace in range(suaWf[unit].shape[1])]))/2:
#             color='red'
#             alpha=1
#         else:
#             color='black'
#             alpha=0.4
#         plt.plot(suaWf[unit][:,trace] - max_amplitude*trace, color=color, alpha=alpha)
#         shift=-5
#         plt.plot(shift,-max_amplitude*trace, 'o', color=color, alpha=alpha)
        
#     plt.text(suaWf.shape[1]/2, max_amplitude/1.5, 'Spike waveforms', ha='center')
#     for trace in [1, suaWf.shape[2]]:
#         plt.text(shift-4, -max_amplitude*(trace-1), f'Channel {trace}', ha='right', va='center')
#     plt.text(shift-7, -max_amplitude*suaWf.shape[2]/2, r'$\vdots$', ha='right', va='center')

#     plt.plot([shift-2, shift-2, shift, shift+2, shift+2, shift-2], 
#         [max_amplitude, -max_amplitude*suaWf.shape[2], -(max_amplitude+5)*suaWf.shape[2], -max_amplitude*suaWf.shape[2],  max_amplitude, max_amplitude],
#         color='k')

#     plt.axis('off')
#     plt.title(f'Unit {unit}, {animal}')
#     plt.xlim(-20,54)
#     plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable


df = pd.read_excel(r'c:\Users\gbouvier\Documents\census.xlsx')
raw_data_path = df['raw_data_path'][df['exp_id'] == animal].iloc[0]

times_polytrodeAll = scipy.io.loadmat(os.path.join(raw_data_path, 'times_polytrodeAll.mat'))
suaWf = times_polytrodeAll['clusWFs']


shift = -5
for unit in range(len(suaWf)):
    fig, ax1 = plt.subplots(figsize=(6, 8))

    
    max_amplitude = max(np.concatenate([abs(suaWf[unit][:, trace]) for trace in range(suaWf[unit].shape[1])]))

    for trace in range(suaWf[unit].shape[1]):
        if np.std(suaWf[unit][:,trace])>np.std(np.concatenate([abs(suaWf[unit][:,trace]) for trace in range(suaWf[unit].shape[1])]))/1.25:
            color='red'
            alpha=1
        else:
            color='black'
            alpha=0.4
        ax1.plot(suaWf[unit][:, trace] - max_amplitude * trace, color=color, alpha=alpha)
        ax1.plot(shift, -max_amplitude * trace, 'o', color=color, alpha=alpha)

    plt.text(suaWf.shape[1]/2, max_amplitude/1.5, 'Spike waveforms', ha='center')
    for trace in [1, suaWf[unit].shape[1]]:
        ax1.text(shift - 4, -max_amplitude * (trace - 1), f'Channel {trace}', ha='right', va='center')
    plt.text(shift-7, -max_amplitude*suaWf.shape[2]/2, r'$\vdots$', ha='right', va='center')
    
    ax1.plot([shift - 2, shift - 2, shift, shift + 2, shift + 2, shift - 2], 
             [max_amplitude, -max_amplitude * suaWf[unit].shape[1], -(max_amplitude + 5) * suaWf[unit].shape[1], 
              -max_amplitude * suaWf[unit].shape[1], max_amplitude, max_amplitude],
             color='k')


    ax1.axis('off')
    ax1.set_xlim(-20, suaWf.shape[1])
    ax1.set_ylim(-(max_amplitude + 6) * suaWf[unit].shape[1], 1.1*max_amplitude)


    divider = make_axes_locatable(ax1)
    ax2 = divider.append_axes("right", size="30%", pad=0.1, sharey=ax1)


    amplitudes_max = np.array([max(abs(suaWf[unit][:, trace])) for trace in range(suaWf[unit].shape[1])])


    im = ax2.imshow(
        amplitudes_max.reshape(-1, 1),
        cmap='OrRd',  
        interpolation='bicubic',
        extent=(0, 1, -max_amplitude * suaWf[unit].shape[1], 0),
        aspect='auto',
        alpha=0.6
    )


    cax = divider.append_axes("right", size="5%", pad=0.1)
    plt.colorbar(im, cax=cax, label=r'Maximal amplitude ($\mu$V)')
    ax2.axis('off')

    fig.suptitle(f'Unit {unit}, {animal}')
    plt.show()

In [ ]:
plt.figure()
plt.plot([0,0,-1,-1.5,-1,1,1.5,1,0], [5,2,-1,-2,-1,-1,-2,-1,2])

plt.show()

In [ ]:
plt.close('all')

# LFP with spikes

In [ ]:
file_path = r'P:\SCANZIANI\ephys 2021\2021\RAW_DATA\21a17\21a17d1s1_210330_130118\amp-A-020.dat'

with open(file_path, 'rb') as file:
    binary_data = file.read()


data = np.frombuffer(binary_data, dtype=np.int16)
data = data*0.195

print(data.shape)

In [ ]:
%matplotlib qt
plt.figure()
plt.plot(rotation_data['animal21_a17d1s1']['time'], data)
plt.show()

In [ ]:
import scipy.signal as signal


for i in range(16,48):
    file_path = rf'P:\SCANZIANI\ephys 2021\2021\RAW_DATA\21a17\21a17d1s1_210330_130118\amp-A-{str(i).zfill(3)}.dat'

    with open(file_path, 'rb') as file:
        binary_data = file.read()


    data = np.frombuffer(binary_data, dtype=np.int16)
    data = data*0.195

    fs = 1000
    t = np.linspace(0, len(data[:1000000]), 1, endpoint=False)


    signal_data = data[:1000000]

    f, t, Sxx = signal.spectrogram(signal_data, fs, nperseg=256, noverlap=128, nfft=512)

    plt.figure()
    plt.pcolormesh(t, f, 10 * np.log10(Sxx), shading='gouraud', cmap='coolwarm')  # Transformation logarithmique pour l'échelle dB
    plt.ylabel('Fréquence [Hz]')
    plt.xlabel('Temps [s]')
    plt.colorbar(label='Intensité [dB]')
    plt.title('Spectrogramme, amp-B-'+str(i-16).zfill(3))
    # plt.ylim(0,100)
    plt.show()


In [ ]:
print(np.load(r'p:\SCANZIANI\ephys 2021\2021\analysed_ephys_2021\animal21_a17d1s1_ephys\21a17d1s1_210330_130118\channel_map.npy')+1)

# 31 is in LP and 3 in hippocampus I think

In [ ]:
# for animal in rotation_data:
#     print(animal, rotation_data[animal]['mapp'], '\n')
#     %matplotlib qt
#     plt.figure(figsize=(5,10))
#     for i in range(len(rotation_data[animal]['mapp'])):
#         plt.text(0, -i, int(rotation_data[animal]['mapp'][i]))
#     plt.xlim(-1,1)
#     plt.ylim(1,-len(rotation_data[animal]['mapp'])-1)
#     plt.axis('off')
#     plt.title('Model H4 '+animal) if animal not in ['animal20_a40d1s1', 'animal20_a41d1s1'] else plt.title('Model H5 '+animal)
#     plt.show()